# Hubbard Model 

In [3]:
using LinearAlgebra
using TensorKit
using MPSKit
using MPSKitModels
using WignerSymbols

## $f\mathbb{Z}_2\otimes U(1)\otimes SU(2)$

In [72]:
elt = ComplexF64
pspace = Vect[fℤ₂ ⊠ U1Irrep ⊠ SU2Irrep]((0, 0, 0) => 1, (1, 1, 1 // 2) => 1, (0, 2, 0) => 1)
vspace = Vect[fℤ₂ ⊠ U1Irrep ⊠ SU2Irrep]((1, 1, 1 // 2) => 1)

e⁺ = TensorMap(zeros, elt, pspace ← pspace ⊗ vspace)
blocks(e⁺)[fℤ₂(0) ⊠ U1Irrep(2) ⊠ SU2Irrep(0)] .= sqrt(2)
blocks(e⁺)[fℤ₂(1) ⊠ U1Irrep(1) ⊠ SU2Irrep(1 // 2)] .= 1
e⁺


TensorMap(Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>1, (0, 2, 0)=>1, (1, 1, 1/2)=>1) ← (Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>1, (0, 2, 0)=>1, (1, 1, 1/2)=>1) ⊗ Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((1, 1, 1/2)=>1))):
* Data for fusiontree FusionTree{(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])}(((0, 2, 0),), (0, 2, 0), (false,), ()) ← FusionTree{(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])}(((1, 1, 1/2), (1, 1, 1/2)), (0, 2, 0), (false, false), ()):
[:, :, 1] =
 1.4142135623730951 + 0.0im
* Data for fusiontree FusionTree{(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])}(((1, 1, 1/2),), (1, 1, 1/2), (false,), ()) ← FusionTree{(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])}(((0, 0, 0), (1, 1, 1/2)), (1, 1, 1/2), (false, false), ()):
[:, :, 1] =
 1.0 + 0.0im


In [89]:
blocks(e⁺)

TensorKit.SortedVectorDict{ProductSector{Tuple{FermionParity, U1Irrep, SU2Irrep}}, Matrix{ComplexF64}} with 2 entries:
  (0, 2, 0)   => [1.41421+0.0im;;]
  (1, 1, 1/2) => [1.0+0.0im;;]

In [114]:
CG = zeros(ComplexF64, 1, 2, 2)
for m1 in -1//2:1//2, m2 in 0:0, m3 in -1//2:1//2
    CG[m2 + 1, Int(m3 + 3//2), Int(m1 + 3//2)] = clebschgordan(0, m2, 1//2, m3, 1//2, m1)
end
CG

1×2×2 Array{ComplexF64, 3}:
[:, :, 1] =
 1.0+0.0im  0.0+0.0im

[:, :, 2] =
 0.0+0.0im  1.0+0.0im

In [115]:
(CG[1,2,2], CG[1,1,1])

(1.0 + 0.0im, 1.0 + 0.0im)

In [116]:
CG = zeros(ComplexF64, 2, 2, 1)
for m1 in -1//2:1//2, m2 in -1//2:1//2, m3 in 0:0
    CG[Int(m1 + 3//2), Int(m2 + 3//2), m3 + 1] = clebschgordan(1//2, m1, 1//2, m2, 0, m3)
end
CG

2×2×1 Array{ComplexF64, 3}:
[:, :, 1] =
      0.0+0.0im  -0.707107+0.0im
 0.707107+0.0im        0.0+0.0im

In [117]:
(CG[2,1,1], CG[1,2,1])

(0.7071067811865476 + 0.0im, -0.7071067811865476 + 0.0im)

In [40]:
P = Vect[fℤ₂ ⊠ U1Irrep ⊠ SU2Irrep]((0, 0, 0) => 1, (1, 1, 1 // 2) => 1, (0, 2, 0) => 1)
V1 = Vect[fℤ₂ ⊠ U1Irrep ⊠ SU2Irrep]((0, 0, 0) => 50, (1, 1, 1 // 2) => 50, (0, 2, 0) => 50, (0, 2, 1) => 10, (1, 3, 1//2) => 10, (0, 4, 0) => 10)
V2 = Vect[fℤ₂ ⊠ U1Irrep ⊠ SU2Irrep]((0, 0, 0) => 10, (1, 1, 1 // 2) => 10, (0, 2, 0) => 10, (0, 2, 1) => 2, (1, 3, 1//2) => 2, (0, 4, 0) => 2)
state = FiniteMPS(12, P, V1);


In [29]:
P = Vect[fℤ₂](0=>2, 1=>2)
V1 = Vect[fℤ₂](0=>200, 1=>200)
L = 12
state = FiniteMPS(L, P, V1);

In [30]:
H = hubbard_model(Float64, FiniteChain(L); t=1, U=0, mu=0);

In [31]:
(gs,env,delta) = find_groundstate(state, H, DMRG2(trscheme=truncbelow(1e-7)));

┌ Info: DMRG2 iteration:
│   iter = 1
│   ϵ = 0.984704459004513
│   λ = -10.947006871482444 - 3.158833733319283e-17im
│   Δt = 288.367181166
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/dmrg.jl:119
┌ Info: DMRG2 iteration:
│   iter = 2
│   ϵ = 1.4765966227514582e-14
│   λ = -10.947006871482444 - 2.7139850128682127e-16im
│   Δt = 67.389106375
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/dmrg.jl:119
┌ Info: DMRG2 summary:
│   ϵ = 1.4765966227514582e-14
│   λ = -10.947006871482444 - 2.7139850128682127e-16im
│   Δt = 356.499427958
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/dmrg.jl:127


In [34]:
sum(expectation_value(gs, H))

-10.947006871482444 - 2.7139850128682127e-16im

In [34]:
(gs,env,delta) = find_groundstate(state, H, VUMPS(; maxiter=400, tol_galerkin=1e-8))

┌ Info: calclw failed to converge 0.005826385579013839
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/environments/mpohaminfenv.jl:97
┌ Info: calcrw failed to converge 0.004119971414567155
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/environments/mpohaminfenv.jl:147
┌ Warning: rightorth failed to converge 1.5195355333505885e-6
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/states/ortho.jl:75
┌ Info: calclw failed to converge 9.640224655043337e-6
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/environments/mpohaminfenv.jl:97
┌ Info: calcrw failed to converge 0.008262448795440426
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/environments/mpohaminfenv.jl:147
┌ Info: VUMPS iteration:
│   iter = 1
│   ϵ = 9.388184800716207e-16
│   λ = 1.205632815803881e-15 + 7.810592450585574e-16im
│   Δt = 193.929540292
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS summary:
│   ϵ = 9.388184

(│   ⋮
│ CR[1]: TensorMap(Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>50, (0, 2, 0)=>50, (0, 4, 0)=>10, (1, 1, 1/2)=>50, (1, 3, 1/2)=>10, (0, 2, 1)=>10) ← Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>50, (0, 2, 0)=>50, (0, 4, 0)=>10, (1, 1, 1/2)=>50, (1, 3, 1/2)=>10, (0, 2, 1)=>10))
├── AL[1]: TensorMap((Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>50, (0, 2, 0)=>50, (0, 4, 0)=>10, (1, 1, 1/2)=>50, (1, 3, 1/2)=>10, (0, 2, 1)=>10) ⊗ Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>1, (0, 2, 0)=>1, (1, 1, 1/2)=>1)) ← Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>50, (0, 2, 0)=>50, (0, 4, 0)=>10, (1, 1, 1/2)=>50, (1, 3, 1/2)=>10, (0, 2, 1)=>10))
│   ⋮
, MPSKit.MPOHamInfEnv{MPOHamiltonian{GradedSpace{ProductSector{Tuple{FermionParity, U1Irrep, SU2Irrep}}, TensorKit.SortedVectorDict{ProductSector{Tuple{FermionParity, U1Irrep, SU2Irrep}}, Int64}}, TensorMap{GradedSpace{ProductSector{Tuple{FermionParity, U1Irrep, SU2Irrep}}, TensorK

In [35]:
expectation_value(gs, H, env)

1-element PeriodicArray{ComplexF64, 1}:
 1.205632815803881e-15 + 7.810592450585574e-16im

In [8]:
P = Vect[fℤ₂](0=>2, 1=>2)
V1 = Vect[fℤ₂](0=>50, 1=>50)
state = InfiniteMPS([P, P], [V1, V1]);

In [9]:
H = hubbard_model(ComplexF64, InfiniteChain(2); t=1, U=1, mu=0);

In [ ]:
(gs,env,delta) = find_groundstate(state, H, IDMRG2(trscheme=truncbelow(1e-5)));

In [ ]:
expectation_value(gs, H, env)

In [6]:
P = Vect[fℤ₂](0=>2, 1=>2)
V1 = Vect[fℤ₂](0=>50, 1=>50)
state = InfiniteMPS([P], [V1]);
H = hubbard_model(ComplexF64, InfiniteChain(1); t=1, U=1, mu=0);

In [7]:
(gs,env,delta) = find_groundstate(state, H, VUMPS(; maxiter=400, tol_galerkin=1e-6))

┌ Info: VUMPS iteration:
│   iter = 1
│   ϵ = 0.38682868173676943
│   λ = -0.356299554389506 + 2.677979366039196e-17im
│   Δt = 1.006620125
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 2
│   ϵ = 0.46246228113260335
│   λ = -0.16317154633910502 - 1.3444106938820255e-17im
│   Δt = 0.86575125
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 3
│   ϵ = 0.49454903680099277
│   λ = 0.036193637124887934 - 3.5887091909270197e-17im
│   Δt = 0.916121833
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 4
│   ϵ = 0.43487972335209496
│   λ = -0.24303022648043304 - 1.242495689668388e-16im
│   Δt = 0.897874916
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 5
│   ϵ = 0.4740640479855

InterruptException: InterruptException: